# Cirq

## Installation

``pip install cirq``

## Funktionstest

In [4]:
import cirq

print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)


## Hello Qubit

In [47]:
# pick a qubit
qubit = cirq.LineQubit(0)

# create a circuit
circuit = cirq.Circuit(
    cirq.H(qubit),
    cirq.measure(qubit, key='m')
)

print("Circuit")
print(circuit)

# simulate
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=50)

print("Result")
print(result)

Circuit
0: ───H───M('m')───
Result
m=10111110010110111000101110101011110101100011101001
